In [1]:
import cv
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras as kr
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


img_height = 299
img_width = 299
batch_size = 32

train_ds = kr.utils.image_dataset_from_directory(
    "/kaggle/input/covid19-radiography-database/Covid-19_Radiography_Dataset/COVID/images",
    seed = 123,
    image_size = (img_height,img_width),
    validation_split = 0.3,
    subset = "training"
)

test_ds = kr.utils.image_dataset_from_directory(
    "/kaggle/input/covid19-radiography-database/Covid-19_Radiography_Dataset/COVID/images",
    seed = 123,
    image_size = (img_height,img_width),
    validation_split = 0.3,
    subset = "validation"
)

def count(counts, batch):
  features, labels = batch
  class_1 = labels == 1
  class_1 = tf.cast(class_1, tf.int32)

  class_0 = labels == 0
  class_0 = tf.cast(class_0, tf.int32)

  class_2 = labels == 2
  class_2 = tf.cast(class_0, tf.int32)
  counts['class_0'] += tf.reduce_sum(class_0)
  counts['class_1'] += tf.reduce_sum(class_1)
  counts['class_2'] += tf.reduce_sum(class_2)

  return counts

counts = train_ds.take(10).reduce(
    initial_state={'class_0': 0, 'class_1': 0, 'class_2' : 0},
    reduce_func = count)

counts = np.array([counts['class_0'].numpy(),
                   counts['class_1'].numpy(),
                   counts['class_2']]).astype(np.float32)

fractions = counts/counts.sum()
print(fractions)




def class_func(features, label):
  return label

resample_ds = (
    train_ds
    .unbatch()
    .rejection_resample(class_func, target_dist=[0.33,0.33,0.33],
                        initial_dist=fractions)
    .batch(32))




model = tf.keras.applications.InceptionResNetV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(img_width,img_height,3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

optimzer = kr.optimizers.Adam(learning_rate=0.0005)
model.compile(
    optimizer="Adam",
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)


early_stop = EarlyStopping(monitor='val_loss', patience=5)

model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=8,
    callbacks=[early_stop],
    shuffle=True
)

print("done")


model.summary()



ModuleNotFoundError: No module named 'cv'